In [15]:
# Imports
import glob
import All_Functions as af
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import random

In [16]:
# Get all the version 2 text files
all_files = [x for x in glob.glob('newspaper-text' + "/*.csv") if "v2" in x]
all_text = af.import_text_data(all_files)

In [17]:
# Put all text into a single df 
compact_list =[all_text[0][0]]
for dataset in all_text: # THERES a memeory problem with using the full dataset at the doc term matrix step!!!!
    for i in range(1,len(dataset)):
        compact_list.append(dataset[i])
        
sample = random.sample(range(1, len(compact_list)), 7000)

sample_list = [compact_list[i] for i in range(len(compact_list)) if i in sample]
print(len(sample_list))

df = pd.DataFrame(data=sample_list, columns=compact_list[0])

7000


In [18]:
"""
Use CountVectorizer to create a document-term matrix 
The paramters mean that we're only including words that appear in less than 80% of the document and in at least 2 documents
Also removing stopwords
"""
count_vect = CountVectorizer(max_df=0.8, min_df=2, stop_words='english')
doc_term_matrix = count_vect.fit_transform(df['text'].values.astype('U'))

In [19]:
'''
This means that each of the 20000 documents is represented as a vector with 
1456 dimension - so our vocabulary has 14546 words'''
doc_term_matrix

<7000x43241 sparse matrix of type '<class 'numpy.int64'>'
	with 2071950 stored elements in Compressed Sparse Row format>

In [20]:
'''
n_components specifics the number of topics that we want our text to be divided into
random_state is the seed so you can replicate your results 
'''
LDA = LatentDirichletAllocation(n_components=4, random_state=42)
LDA.fit(doc_term_matrix)

LatentDirichletAllocation(n_components=4, random_state=42)

In [21]:
for i,topic in enumerate(LDA.components_):
    print(f'Top 10 words for topic #{i}:')
    print([count_vect.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')

Top 10 words for topic #0:
['photo', 'said', 'sunday', 'music', 'paddock', 'festival', '2017', 'people', 'las', 'vegas']


Top 10 words for topic #1:
['time', 'trump', 'year', 'going', 'state', 'said', 'new', 'like', 'gun', 'people']


Top 10 words for topic #2:
['white', 'time', 'texas', 'violence', 'news', 'president', 'people', 'trump', 'gun', 'said']


Top 10 words for topic #3:
['time', 'news', 'ufc', 'las', 'school', 'told', 'vegas', 'police', 'family', 'said']




In [37]:
topic_values = LDA.transform(doc_term_matrix)
'''output means that each of the documents have 5 columns where each column 
corresponds to the probabilitiy value of a particular topic'''
topic_values.shape

(7000, 6)

In [38]:
df['Topic'] = topic_values.argmax(axis=1)
df.head()

,,url,text,sentiment,publish_date,title,themes,media_id,media_url,Leaning,Location,Topic
0,3,http://feedproxy.google.com/~r/time/topstories...,least five people killed monday disgruntled em...,"{'neg': 0.224, 'neu': 0.74, 'pos': 0.036, 'com...",2017-06-05 11:46:24,‘Disgruntled’ Ex-Employee Kills 5 in Orlando S...,,40362,http://www.time.com/time/,Bogue,None,4
1,6,http://www.fark.com/comments/9617590?utm_sourc...,iczer ca wait see twit chief say ... marcus au...,"{'neg': 0.192, 'neu': 0.602, 'pos': 0.206, 'co...",2017-06-07 03:33:23,Mass shooting reported (spins wheel) inside Ir...,,19921,http://www.fark.com,Bogue,None,2
2,10,http://feeds.chicagotribune.com/~r/chicagotrib...,family members crowded small corridor emergenc...,"{'neg': 0.161, 'neu': 0.828, 'pos': 0.011, 'co...",2017-06-11 11:34:00,2 injured on Riverwalk and 9 in Lawndale among...,,9,http://www.chicagotribune.com/,center,Bogue,4
3,16,http://www.startribune.com/local/427850713.html,rainbows dotted metro area sunday despite gray...,"{'neg': 0.079, 'neu': 0.77, 'pos': 0.152, 'com...",2017-06-11 22:19:34,"Pride rally, festival in Minn. carry theme of ...",,19,http://www.startribune.com/,Bogue,None,2
4,17,http://network.yardbarker.com/nba/article_exte...,free newsletters facebook twitter flipboard yo...,"{'neg': 0.028, 'neu': 0.818, 'pos': 0.154, 'co...",2017-06-12 01:23:50,Orlando found its identity in the year after P...,,86839,http://www.yardbarker.com/#spider,center,Bogue,5


In [39]:
# source: https://towardsdatascience.com/evaluate-topic-model-in-python-latent-dirichlet-allocation-lda-7d57484bb5d0
from gensim.models import CoherenceModel
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=LDA, texts=doc_term_matrix, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

AttributeError: 'LatentDirichletAllocation' object has no attribute 'id2word'

In [40]:
# same source4
import gensim
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(sample_list)
# Create Corpus
texts = sample_list
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1])
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=sample_list, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1)]]


In [41]:
import numpy as np
import tqdm
grid = {}
grid['Validation_Set'] = {}
# Topics range
min_topics = 2
max_topics = 11
step_size = 1
topics_range = range(min_topics, max_topics, step_size)
# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')
# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')
# Validation sets
num_of_docs = len(corpus)
corpus_sets = [# gensim.utils.ClippedCorpus(corpus, num_of_docs*0.25), 
               # gensim.utils.ClippedCorpus(corpus, num_of_docs*0.5), 
               gensim.utils.ClippedCorpus(corpus, num_of_docs*0.75), 
               corpus]
corpus_title = ['75% Corpus', '100% Corpus']
model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }
print(model_results)

# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=540)
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            print(k)
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
#                     cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, 
#                                                   k=k, a=a, b=b)
                    cv = compute_coherence_values(corpus=corpus, dictionary=id2word, k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv('lda_tuning_results.csv', index=False)
    pbar.close()

  0%|          | 0/540 [00:00<?, ?it/s]

{'Validation_Set': [], 'Topics': [], 'Alpha': [], 'Beta': [], 'Coherence': []}
2


  6%|▌         | 30/540 [16:55<4:59:32, 35.24s/it]

3


 11%|█         | 60/540 [34:14<3:43:50, 27.98s/it]

...

 89%|████████▉ | 480/540 [6:59:13<27:58, 27.98s/it]

9


 94%|█████████▍| 510/540 [7:15:43<16:30, 33.00s/it]

10


100%|██████████| 540/540 [7:31:15<00:00, 50.14s/it]


In [42]:
model_results = {'Topics':[]}

In [43]:
model_results['Topics'].append('helo')

In [44]:
model_results

{'Topics': ['helo']}

In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('lda_tuning_results.csv')

,Validation_Set,Topics,Alpha,Beta,Coherence
0,75% Corpus,2,0.01,0.01,0.522850
1,75% Corpus,2,0.01,0.31,0.375227
2,75% Corpus,2,0.01,0.61,0.371268
3,75% Corpus,2,0.01,0.9099999999999999,0.372665
4,75% Corpus,2,0.01,symmetric,0.376639


In [8]:
df[df['Coherence'] ==df['Coherence'].max()]

,Validation_Set,Topics,Alpha,Beta,Coherence
85,75% Corpus,4,asymmetric,0.01,0.572947


In [11]:
import gensim
import gensim.corpora as corpora

In [12]:
# Get all the version 2 text files
all_files = [x for x in glob.glob('newspaper-text' + "/*.csv") if "v2" in x]
all_text = af.import_text_data(all_files)

# Put all text into a single df 
compact_list =[all_text[0][0]]
for dataset in all_text: # THERES a memeory problem with using the full dataset at the doc term matrix step!!!!
    for i in range(1,len(dataset)):
        compact_list.append(dataset[i])

In [14]:
sample = random.sample(range(1, len(compact_list)), 7000)

sample_list = [compact_list[i] for i in range(len(compact_list)) if i in sample]
print(len(sample_list))

df = pd.DataFrame(data=sample_list, columns=compact_list[0])

# Create Dictionary
id2word = corpora.Dictionary(sample_list)
# Create Corpus
texts = sample_list
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]


lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=7, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha='asymmetric',
                                           eta=0.01)

7000


In [24]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

ModuleNotFoundError: No module named 'pyLDAvis'

In [23]:
pip install pyLDAvis

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: setuptools in c:\users\khahn\anaconda3\lib\site-packages (from pyLDAvis) (52.0.0.post20210125)
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136897 sha256=beb3420486b6eda11285d4df910de3a70c9c4c5c6ebc0d844241917ffb94e0b0
  Stored in directory: c:\users\khahn\appdata\local\pip\cache\wheels\90\61\ec\9dbe9efc3acf9c4e37ba70fbbcc3f3a0ebd121060aa593181a
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1316 sha256=66ff71e0c10c402

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\khahn\\anaconda3\\Lib\\site-packages\\~-mpy\\.libs\\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll'
Consider using the `--user` option or check the permissions.



In [ ]:
# Get all the version 2 text files
all_files = [x for x in glob.glob('relevancy-files' + "/*.csv")]
# all_text = af.import_text_data(all_files)

In [ ]:
def run_LDA(file):
    df = pd.read_csv(file)
    relevant_df = df[df['Relevancy']==1]
    print(len(relevant_df))
    count_vect = CountVectorizer(max_df=0.8, min_df=2, stop_words='english')
    doc_term_matrix = count_vect.fit_transform(relevant_df['text'].values.astype('U'))
    LDA = LatentDirichletAllocation(n_components=4, random_state=42)
    LDA.fit(doc_term_matrix)

    for i, topic in enumerate(LDA.components_):
        print(f'Top 10 words for topic #{i+1}:')
        print([count_vect.get_feature_names()[i] for i in topic.argsort()[-10:]])
        print('\n')

    topic_values = LDA.transform(doc_term_matrix)
    relevant_df['Topic'] = topic_values.argmax(axis=1)

    pyLDAvis.sklearn.prepare(LDA, doc_term_matrix, count_vect)
    return df.values.tolist()



In [ ]:
boulder = run_LDA(all_files[1])